# 猫粮配比问题
### 模型一：只考虑鸡肉、牛肉成分

In [1]:
# 导入pulp包并定义为*
from pulp import *

# 定义一个名为 "The Whiskas Problem" 的线性规划问题prob
prob = LpProblem("The Whiskas Problem", LpMinimize)

# 决策变量
x1 = LpVariable("鸡肉", 0, None, LpInteger)
x2 = LpVariable("牛肉", 0, None, LpInteger)

# 目标函数
prob += 0.013 * x1 + 0.008 * x2, "Total Cost of Ingredients per can"

# 约束条件
# prob +=：表示向现有的线性规划问题 prob 中添加一个新的约束
prob += x1 + x2 == 100, "PercentagesSum"
prob += 0.100 * x1 + 0.200 * x2 >= 8.0, "ProteinRequirement"
prob += 0.080 * x1 + 0.100 * x2 >= 6.0, "FatRequirement"
prob += 0.001 * x1 + 0.005 * x2 <= 2.0, "FibreRequirement"
prob += 0.002 * x1 + 0.005 * x2 <= 0.4, "SaltRequirement"

# 将定义好的线性规划问题写入.lp文件
prob.writeLP("WhiskasModel.lp")

# 使用pulp包的求解器求解
#不需要括号中的参数，因为 PuLP 会使用它内部的默认设置
prob.solve() 

# 输出解决方案的状态
print("Status:", LpStatus[prob.status])

# 输出每个决策变量的最优解值
for v in prob.variables(): 
    print(v.name, "=", v.varValue)
#使用variables()方法遍历并返回的所有决策变量
#v.varValue决策变量的最优值，即求解器找到的该变量在最优解中的值

# 输出最优解
print("每罐配料的总成本 = ", value(prob.objective))


Status: Optimal
牛肉 = 66.0
鸡肉 = 34.0
每罐配料的总成本 =  0.97


D:\Anaconda3\lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


### 模型二：考虑全部成分

In [2]:
from pulp import *

# 对猫粮组成成分以名称创建一个列表
Ingredients = ["CHICKEN", "BEEF", "MUTTON", "RICE", "WHEAT","GEL"]

# 数据输入
# 创建每类数据的字典
# cost
costs = {
    "CHICKEN": 0.013,
    "BEEF": 0.008,
    "MUTTON": 0.010,
    "RICE": 0.002,
    "WHEAT": 0.005,
    "GEL": 0.001}

# proteinPercent
proteinPercent = {
    "CHICKEN": 0.100,
    "BEEF": 0.200,
    "MUTTON": 0.150,
    "RICE": 0.000,
    "WHEAT": 0.040,
    "GEL": 0.000}

# fatPercent
fatPercent = {
    "CHICKEN": 0.080,
    "BEEF": 0.100,
    "MUTTON": 0.110,
    "RICE": 0.010,
    "WHEAT": 0.010,
    "GEL": 0.000}

# fibrePercent
fibrePercent = {
    "CHICKEN": 0.001,
    "BEEF": 0.005,
    "MUTTON": 0.003,
    "RICE": 0.100,
    "WHEAT": 0.150,
    "GEL": 0.000}

# saltPercent
saltPercent = {
    "CHICKEN": 0.002,
    "BEEF": 0.005,
    "MUTTON": 0.007,
    "RICE": 0.002,
    "WHEAT": 0.008,
    "GEL": 0.000}


prob = LpProblem("The Whiskas Problem", LpMinimize)

# 创建一个名为 "inquestient_vars "的字典，包含引用的变量
# 创建多个 LpVariable 实例，并以字典的形式返回它们
ingredient_vars = LpVariable.dicts("Ingr", Ingredients, 0)
# 变量名会显示为 Ingr_IngredientName

# 目标函数
prob += (
    lpSum([costs[i] * ingredient_vars[i] for i in Ingredients]),
    "Total Cost of Ingredients per can",
)

# 约束条件
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += (
    lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0,
    "ProteinRequirement")
prob += (
    lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0,
    "FatRequirement")
prob += (
    lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0,
    "FibreRequirement")
prob += (
    lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4,
    "SaltRequirement")


prob.writeLP("WhiskasModel2.lp")
prob.solve()

print("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)

print("每罐配料的总成本 = ", value(prob.objective))


Status: Optimal
Ingr_BEEF = 60.0
Ingr_CHICKEN = 0.0
Ingr_GEL = 40.0
Ingr_MUTTON = 0.0
Ingr_RICE = 0.0
Ingr_WHEAT = 0.0
每罐配料的总成本 =  0.52


D:\Anaconda3\lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


# 啤酒分销问题:供需平衡

In [11]:
from pulp import *

# 创建仓库的列表
Warehouses = ["A", "B"]

# 为每个供应节点创建一个供应单位数量字典
supply = {"A": 1000, "B": 4000}

# 创建所有需求节点的列表
Bars = ["1", "2", "3", "4", "5"]

# 为每个需求节点创建一个需求单位数量字典
demand = {
    "1": 500,
    "2": 900,
    "3": 1800,
    "4": 200,
    "5": 700}

# 创建每个运输路径的成本列表
# 行表示仓库，列表示酒吧，数值表示仓库-酒吧的单位成本
costs = [  # Bars 1 2 3 4 5
    [2, 4, 5, 2, 1],  # A   Warehouses
    [3, 1, 3, 2, 3],  # B
]

# 将成本数据编入字典
costs = makeDict([Warehouses, Bars], costs, 0)


prob = LpProblem("Beer Distribution Problem", LpMinimize)

# 创建一个包含所有可能运输路线索引的列表
Routes = [(w, b) for w in Warehouses for b in Bars]


# 创建一个名为 "vars "的字典，其中包含引用的变量（Route）
vars = LpVariable.dicts("Route", (Warehouses, Bars), 0, None, LpInteger)
# LpVariable.dicts("Route",(Warehouses, Bars),lowBound=0,upBound=None,cat='LpInteger')
# name：变量前缀; index是列表，其中的元素会被用来构成变量名


# 约束条件
prob += (
    lpSum([vars[w][b] * costs[w][b] for (w, b) in Routes]),
    "Sum_of_Transporting_Costs",
)

# 每个供应节点（仓库）的供应最大约束条件
for w in Warehouses:
    prob += (
        lpSum([vars[w][b] for b in Bars]) <= supply[w],
        f"Sum_of_Products_out_of_Warehouse_{w}",
    )

# 每个需求节点（酒吧）的需求最小约束条件
for b in Bars:
    prob += (
        lpSum([vars[w][b] for w in Warehouses]) >= demand[b],
        f"Sum_of_Products_into_Bar{b}",
    )


prob.writeLP("BeerDistributionProblem.lp")
prob.solve()


print("Status:", LpStatus[prob.status])


for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Total Cost of Transportation = ", value(prob.objective))

Status: Optimal
Route_A_1 = 300.0
Route_A_2 = 0.0
Route_A_3 = 0.0
Route_A_4 = 0.0
Route_A_5 = 700.0
Route_B_1 = 200.0
Route_B_2 = 900.0
Route_B_3 = 1800.0
Route_B_4 = 200.0
Route_B_5 = 0.0
Total Cost of Transportation =  8600.0


D:\Anaconda3\lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## 扩展模型1：需<供
### 需求小于供应——设置虚拟需求节点

In [12]:
## 需求小于供应——设置虚拟需求节点
from pulp import *

# 数据准备
Warehouses = ["A", "B"]
supply = {"A": 1000, "B": 4000}
Bars = ["1", "2", "3", "4", "5", "D"]
demand = {"1": 500, "2": 900, "3": 1800, "4": 200, "5": 700, "D": 900}
costs = [  # Bars
    # 1 2 3 4 5 D
    [2, 4, 5, 2, 1, 0],  # A   Warehouses
    [3, 1, 3, 2, 3, 0],  # B
]
costs = makeDict([Warehouses, Bars], costs, 0)


# 模型求解
prob = LpProblem("Beer Distribution Problem", LpMinimize)

Routes = [(w, b) for w in Warehouses for b in Bars]

# 决策变量
vars = LpVariable.dicts("Route", (Warehouses, Bars), 0, None, LpInteger)

# 目标函数
prob += (
    lpSum([vars[w][b] * costs[w][b] for (w, b) in Routes]),
    "Sum_of_Transporting_Costs",
)

# 约束条件
for w in Warehouses:
    prob += (
        lpSum([vars[w][b] for b in Bars]) <= supply[w],
        f"Sum_of_Products_out_of_Warehouse_{w}",
    )

for b in Bars:
    prob += (
        lpSum([vars[w][b] for w in Warehouses]) >= demand[b],
        f"Sum_of_Products_into_Bar{b}",
    )


prob.writeLP("BeerDistributionProblem.lp")
prob.solve()

print("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Total Cost of Transportation = ", value(prob.objective))

Status: Optimal
Route_A_1 = 300.0
Route_A_2 = 0.0
Route_A_3 = 0.0
Route_A_4 = 0.0
Route_A_5 = 700.0
Route_A_D = 0.0
Route_B_1 = 200.0
Route_B_2 = 900.0
Route_B_3 = 1800.0
Route_B_4 = 200.0
Route_B_5 = 0.0
Route_B_D = 900.0
Total Cost of Transportation =  8600.0


D:\Anaconda3\lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


# 扩展模型2:需>供
### 需求大于供应——设置虚拟供应节点

In [13]:
## 需求大于供应——设置虚拟供应节点
from pulp import *

Warehouses = ["A", "B", "C"]
supply = {"A": 1000, "B": 4000, "C": 100}
Bars = ["1", "2", "3", "4", "5"]
demand = {
    "1": 500,
    "2": 900,
    "3": 1800,
    "4": 200,
    "5": 700,
}
costs = [  # Bars
    # 1 2 3 4 5
    [2, 4, 5, 2, 1],  # A   Warehouses
    [3, 1, 3, 2, 3],  # B
    [0, 0, 0, 0, 0],
]

costs = makeDict([Warehouses, Bars], costs, 0)

prob = LpProblem("Beer Distribution Problem", LpMinimize)

Routes = [(w, b) for w in Warehouses for b in Bars]

# 决策变量
vars = LpVariable.dicts("Route", (Warehouses, Bars), 0, None, LpInteger)

# 目标函数
prob += (
    lpSum([vars[w][b] * costs[w][b] for (w, b) in Routes]),
    "Sum_of_Transporting_Costs",
)

# 约束条件
for w in Warehouses:
    prob += (
        lpSum([vars[w][b] for b in Bars]) <= supply[w],
        f"Sum_of_Products_out_of_Warehouse_{w}",
    )

for b in Bars:
    prob += (
        lpSum([vars[w][b] for w in Warehouses]) >= demand[b],
        f"Sum_of_Products_into_Bar{b}",
    )

prob.writeLP("BeerDistributionProblem.lp")
prob.solve()

print("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Total Cost of Transportation = ", value(prob.objective))

Status: Optimal
Route_A_1 = 300.0
Route_A_2 = 0.0
Route_A_3 = 0.0
Route_A_4 = 0.0
Route_A_5 = 700.0
Route_B_1 = 200.0
Route_B_2 = 900.0
Route_B_3 = 1700.0
Route_B_4 = 200.0
Route_B_5 = 0.0
Route_C_1 = 0.0
Route_C_2 = 0.0
Route_C_3 = 100.0
Route_C_4 = 0.0
Route_C_5 = 0.0
Total Cost of Transportation =  8300.0


D:\Anaconda3\lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
